In [11]:
#Packages being downloaded
import csv
import pandas as pd
import re
import xml.etree.ElementTree as ET

from tkinter import filedialog
from tkinter import *

import os

In [10]:
def getFile(sdtFile):
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("text files","*.csv"),("all files","*.*")))
    changePath(root.filename,sdtFile)
    root.destroy()
    return(root.filename)

In [9]:
def changePath(path,sdtFile):
    tree = ET.parse(sdtFile)
    root = tree.getroot()
    
    for elem in root.findall('.//Datasources/Datasource/Host'):
        elem.text=path
    
    for elem in root.findall('.//Datasources/Datasource/Name'):
        elem.text=path
        
    tree.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [8]:
def readCSV(csvName):
    df = pd.read_csv(csvName)
    print("reading csv done")
    return df

In [7]:
def getCat(df,colorName):
    x=0
    for name in df.columns:
        if df.dtypes[x] == 'object':
             if len(df[name].astype('category').cat.categories.tolist()) >1 and len(df[name].astype('category').cat.categories.tolist()) <101 and colorName=="": 
                colorName=df.columns[x]
        x= x+1
    return colorName

In [5]:
def getCatCol(df,colorName):
    return df[colorName].astype('category').cat.categories.tolist()

In [4]:
def typeDefiner(df,catFound,sdtFile,xName,yName):
    colList = df.columns
    colList
    x = 0
    colNames = []
    for name in colList:
        if df.dtypes[x] == 'object' and colList[x] is xName or colList is yName:# or df.dtypes[x] == 'int64':
            pass
        else:
            colNames.append(colList[x])
                
        x = x + 1
    
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputCount=0
    for elem in root.findall('.//InputFields'):
        for inputs in elem.findall('.//InputField'):
            inputCount+=1
    
    if not catFound and inputCount>len(colNames):
        resetToDefault(sdtFile)
    print("input count is "+str(catFound)+str(len(colNames)))
    print("objects removed")
    print(colNames)
    return colNames

In [3]:
def toNumeric(sdtFile):

    #import xml.etree.ElementTree as ET
    #CLEARS COLOR OUT NOW WE CAN INSERT ANYTHING
    file = ET.parse(sdtFile)
    root = file.getroot()
    for elem in root.findall('.//Glyph/Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
        elem.clear()
        

    for parent in root.findall('.//KeyValuePairs/..'):
        for element in parent.findall('KeyValuePairs'):
            parent.remove(element)
            
    
   #Try to write here
    for elem in root.findall('.//Glyph/Color/RGB/..'):
        check_req_elems = elem.find('.//Function')
        #check_req_elems.type = "Linear Interpolation"
        check_req_elems.set("type","Linear Interpolation")
        attrib = {'type': 'BoundField'}
        minmax = ET.SubElement(check_req_elems, 'MinMax', attrib)
        minmax.text= "\n\t\t\t\t\t\t\t"
        min = ET.SubElement(minmax, 'Min')
        min.text = '0'
        min.tail="\n\t\t\t\t\t\t"
        minmax.tail="\n\t\t\t\t\t"
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)
    print("color fixed")

In [28]:
def XMLParser(colNames, sdtFile,colorName,catFound,dict,xName,xFound,yName,yFound):
    print(colNames)

    tree = ET.parse(sdtFile)
    root = tree.getroot()
    count=0;
    print(len(colNames))
    for elem in root.iter('InputField'):
        print(count)
        if elem.get('name')==dict['ColorRGB'] and catFound:
            elem.set('field', colorName)
        elif elem.get('name')==dict['PositionX'] and xFound:
            elem.set('field', xName)
        elif elem.get('name')==dict['PositionY'] and yFound:
            elem.set('field', yName)
        else:
            elem.set('field', colNames[count])
            count+=1
            
        
    #tree.write('New DAL 900.sdt')
    tree.write(sdtFile)
    print("XML updated")

In [42]:
def runAll(sdtFile):
    colorCol= None
    colorName=""
    catFound=False
    
    xName=''
    xFound=False
    
    yName=''
    yFound=False
    
    csv = getFile(sdtFile)
    df = readCSV(csv)
    colorName=getCat(df,colorName)
    if(colorName!=""):
        catFound=True
        colorCol=getCatCol(df,colorName)
    
    #Find x
    xName=latFinder2(df)
    if(xName!=""):
        xFound=True
        #Don't need a list of value pairs for x, y, or z
            
    #Find y 
    yName=longFinder2(df)
    if(yName!=""):
        yFound=True
    
    sdtFile=moveToNewFile(sdtFile,csv)
    colNames = typeDefiner(df,catFound,sdtFile,xName,yName)
    #colNames.append(colorName)
    dict =MakeDict('emptySDT.sdt')
   
    XMLParser(colNames, sdtFile,colorName,catFound,dict,xName,xFound,yName,yFound)
    setupColor(colorCol, sdtFile)
   # csv = getFile()
#readCSV(csv)

In [78]:
#%%python
sevenTabs="\n\t\t\t\t\t\t\t"
eightTabs="\n\t\t\t\t\t\t\t\t"
sixTabs="\n\t\t\t\t\t\t"
runAll('emptySDT.sdt')

reading csv done
input count is True23
objects removed
['id', 'date', 'dateTime', 'Hitter', 'Hitter_Team', 'Pitcher', 'Pitcher_Team', 'Inning', 'Ballpark', 'Type_Luck', 'True_Distance', 'Speed_off_Bat', 'Elevation_Angle', 'Horizontal_Angle', 'Apex', 'Angle_Adj', 'Location_X', 'Location_Y', 'Field_Location', 'Nametag_HR', 'year', 'month', 'season_type']
['id', 'date', 'dateTime', 'Hitter', 'Hitter_Team', 'Pitcher', 'Pitcher_Team', 'Inning', 'Ballpark', 'Type_Luck', 'True_Distance', 'Speed_off_Bat', 'Elevation_Angle', 'Horizontal_Angle', 'Apex', 'Angle_Adj', 'Location_X', 'Location_Y', 'Field_Location', 'Nametag_HR', 'year', 'month', 'season_type']
23
0
0
0
1
XML updated
Setting up Colors


# Extra functions

In [4]:
def elevationFinder(df):
    if df.columns.contains("elevation") or df.columns.contains("Elevation") or df.columns.contains("height"):
        longitude = df["Longitude"]
    else:
        longitude = "none found"
    return longitude

In [75]:
def latFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'^LAT', df.columns[x], re.I)
        searchObj2= re.match( r'.*[\s_\b]X.*', df.columns[x], re.I)
        #print(str(df.columns[x]) + str(searchObj2))
        if searchObj or searchObj2:
            #print("found it")
            lat = df.columns[x]
            return lat
        else:
            x = x + 1
            pass
    return ""

In [77]:
def longFinder2(df):
    x = 0
    for columns in df.columns:
        searchObj = re.match( r'^LONG', df.columns[x], re.I)
        searchObj2= re.match( r'.*[\s_\b]Y.*', df.columns[x], re.I)
        if searchObj or searchObj2:
            #print("found it")
            long = df.columns[x]
            return long
        else:
            x = x + 1
            pass
    return ""

In [16]:
def setupColor(colorCol,sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    x=0
    if(colorCol==None):#and then potentially add a binding field
        print("Changing to numeric")
        toNumeric(sdtFile)
        
    else:
        print("Setting up Colors")
        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
            for element in elem.findall('KeyValuePair/Key'):
                element.text=colorCol[x]
                x=x+1

            if(len(colorCol)>2):
                while x<len(colorCol):
                    for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]'):
                        pairs = elem.find('.//KeyValuePairs')
                #fixes spacing so new pair will be at right column/indent area
                        for child in pairs.findall('.//KeyValuePair'):
                            child.tail=sevenTabs
                        keyVal =ET.SubElement(pairs, 'KeyValuePair')
                        keyVal.text=eightTabs
                        keyVal.tail=sixTabs
                        keyValP= ET.SubElement(keyVal, 'Key')
                        keyValP.text=colorCol[x]
                        keyValP.tail=eightTabs
                        key = ET.SubElement(keyVal, 'Value')
                    
                        key.tail=sevenTabs
                        x=x+1
    
                        x=0

                        count=0
                        colorList=[] 
                        while count<len(colorCol):
                            randomCol(colorList)
                            count=count+1


                        for elem in root.findall('.//Color/RGB/Function[@type="Text Field To Value"]/KeyValuePairs'):
                            for element in elem.findall('KeyValuePair/Value'):
                                element.text=colorList[x]
                                x=x+1
                
                
            file.write(sdtFile,encoding="utf-8", xml_declaration=True)


In [17]:
#
def moveToNewFile(sdtFile,csv):
    file = ET.parse(sdtFile)
    base=os.path.basename(csv)
    newFile =os.path.splitext(base)[0]+".sdt"
    file.write(newFile,encoding="utf-8", xml_declaration=True)
    
    return newFile

In [18]:
def randomCol(colorList):
    import random
    r = random.random()*255
    r=int(r)
    g = random.random()*255
    g=int(g)
    b = random.random()*255
    b=int(b)
    color=str(r)+","+str(g)+","+str(b)
    colorList.append(color)

In [19]:
def resetToDefault(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    print("resetting")
    element=None
    for parent in root.findall('.//Glyph/Color/RGB'):
        for e in parent:
            print(e)
            if(element==None):
                element = parent.find('Binding')
                print(element)
        if(element!=None):
            parent.remove(element)
            element=None
    
    for parent in root.findall('.//InputFields'):
        for e in parent:
            print(e)
            if(element==None):
                element =  parent.find('InputField[@name="color"]')
                print(element)
                print("Element found")
        if (element!=None):
            parent.remove(element)
            element=None
    
    count=0
    for parent in root.findall('.//InputFields'):
        for e in parent:
            if(count==len(parent)-1):
                e.tail="\n\t"
            count+=1
        
    file.write(sdtFile,encoding="utf-8", xml_declaration=True)

In [20]:
def MakeDict(sdtFile):
    file = ET.parse(sdtFile)
    root = file.getroot()
    inputs={'Name':'Id'}
    for parent in root.findall('.//Glyph'):
        for elem in parent:
            for e in elem:
                for l in e:
                    temp=e.find('Binding')
                    if temp!=None:
                        inputs[elem.tag+e.tag]=temp.get('fieldId')
                        temp=None
                        
    return inputs

In [18]:
dict =MakeDict('emptySDT.sdt')
print(dict)

{'Name': 'Id', 'PositionX': '101', 'PositionY': '111', 'PositionZ': 'z', 'ColorRGB': '555'}
